# 切出原告主張(修正PK與建立錯誤log檔案)

In [40]:
import os 
import re
import codecs
import _uniout 
import glob
import pymongo
from pymongo import MongoClient
import traceback

def no_court(content):
    try:
        court_no = {u"臺北地方":100,u"士林地方":111,u"新北地方":220,u"宜蘭地方":260,u"基隆地方":200,u"桃園地方":320,u"新竹地方":300,u"苗栗地方":350,u"臺中地方":400,u"彰化地方":500,u"南投地方":540,u"臺南地方":700,u"高雄地方":800,u"花蓮地方":970,u"雲林地方":630,u"嘉義地方":602,u"臺東地方":950,u"屏東地方":900,u"澎湖地方":880,u"金門地方":890,u"連江地方":209,u"高雄少年及家事":850}
        m = re.search(u'[  \d]*(灣|福建|臺灣)(\S{4,7})法院(民事|家事|暫時|支付)(\S{2})',content)
        return str(court_no[m.group(2)])
    except:
        print "Rex Err:"+m.group(2)
def no_jurytype(content):
    try:
        jurytype_no = {u"婚":11,u"婚再":12,u"婚更":13,u"家聲":21,u"家訴":22,u"家抗":23,u"家陸許":24,u"司家聲":30,u"基家簡":25,u"重家訴":26,u"婚更一":27,u"婚更(一)":28,u"司家調":29,u"家移調":31,u"家簡":32,u"家調裁":33,u"家查":34,u"家聲抗":35,u"家調裁":36,u"家調補":37,u"司家補":38,u"家親聲":39,u"司家簡調":40,u"家簡上":41,u"家小":42,u"家救":43,u"家小上":44,u"家婚聲":45,u"家再":46,u"家他":47,u"訴":48,u"重訴":49,u"家續":50,u"審訴":51,u"聲":52,u"補":90}
        m = re.search(u'(【裁判字號】)(.*?),(.*?),(\d{1,4})[^【】 ]',content)
        return m.group(2)+"_"+str(jurytype_no[m.group(3)])+"_"+m.group(4)
    except:
        print "Rex Err:"+m.group(2)
def judgedate(content):
    m = re.search(u'(【裁判日期】)(\d{6,7})',content)
    return m.group(2)

def getPK(content):
    return no_court(content)+"_"+no_jurytype(content)+"_"+judgedate(content)

#make a connection to mongodb 
client = pymongo.MongoClient('localhost',27017)
#assign db_name to db
db = client.test
#make the collection in the db of test
collection = db['plaintiff']
plaintiff = db.plaintiff
#read the judicial case in the directory

list0 = []
list1 = [u'壹',u'貳',u'參',u'肆']
list2 = [u'一',u'二',u'三',u'四']
list3 = [u'甲',u'乙',u'丙',u'丁']
for filename in glob.glob(u'E:\\Dropbox\\2015bigdata\\riii\\rename_judgement\\*.txt'):
    with codecs.open( filename ,'r' ,'utf-8') as f :
        content = f.read() # 不能圖超過 2Mb        
        f.seek(0,0) #檔案指標移回檔案開頭
        lines = f.readlines()# 不能圖超過 2Mb
        cnt = 0
        lv = 0
        txt = u''
        #print lines[cnt]
        try:
            for i in lines:
                if lv != 0 or cnt > 70:
                    break
                m = re.search(u'([壹貳參肆一二三四甲乙丙丁]{1,2})、(原告.*?)',i)
                if m !=None:
                    n = m.group(1)
                    if n in list1:
                        lv = 1
                    if n in list2:
                        lv = 2
                    if n in list3:
                        lv = 3
                    for j in lines:
                        txt += lines[cnt]
                        cnt += 1
                        if lv == 1 and re.match(u'[壹貳參肆]{1,2}、.*?',lines[cnt]) !=None:
                            break
                        if lv == 2 and re.match(u'[一二三四]{1,2}、.*?',lines[cnt]) !=None:
                            break
                        if lv == 3 and re.match(u'[甲乙丙丁]{1,2}、.*?',lines[cnt]) !=None:
                            break
                else:
                    cnt += 1            
        except:
            f = open("./log/log_cut_plaintiff.txt",'a+')
            log = "cuterr:"+"\t"+ filename +"\n"
            f.write(log)
            f.close()
            print 'CutErr:'+traceback.format_exc() #顯示詳細錯誤訊息
            pass
        dic = {"_id":getPK(content),"juryno":no_jurytype(content),"judgedate":judgedate(content),"plaintiff":txt}
        #list0.append(getPK(content))
        try:
            #insert to the collection of dday
            dday_id = plaintiff.insert(dic)
        except:
            f2 = open("./log/log_mongodb_plaintiff.txt",'a+')
            log2 = "mongoerr:"+"\t"+ filename +"\n"
            f2.write(log2)
            f2.close()
            print 'MongoErr:'+traceback.format_exc() #顯示詳細錯誤訊息
            pass

CutErr:Traceback (most recent call last):
  File "<ipython-input-40-711ae7f7b03e>", line 73, in <module>
    if lv == 3 and re.match(u'[甲乙丙丁]{1,2}、.*?',lines[cnt]) !=None:
IndexError: list index out of range

CutErr:Traceback (most recent call last):
  File "<ipython-input-40-711ae7f7b03e>", line 71, in <module>
    if lv == 2 and re.match(u'[一二三四]{1,2}、.*?',lines[cnt]) !=None:
IndexError: list index out of range

CutErr:Traceback (most recent call last):
  File "<ipython-input-40-711ae7f7b03e>", line 71, in <module>
    if lv == 2 and re.match(u'[一二三四]{1,2}、.*?',lines[cnt]) !=None:
IndexError: list index out of range

CutErr:Traceback (most recent call last):
  File "<ipython-input-40-711ae7f7b03e>", line 71, in <module>
    if lv == 2 and re.match(u'[一二三四]{1,2}、.*?',lines[cnt]) !=None:
IndexError: list index out of range

CutErr:Traceback (most recent call last):
  File "<ipython-input-40-711ae7f7b03e>", line 71, in <module>
    if lv == 2 and re.match(u'[一二三四]{1,2}、.*?',lines[cnt]) 

# 顯示詳細的ERR訊息

In [36]:
import glob
import traceback
list = [1,3,4,5]
#for filename in glob.glob(u'E:\\Dropbox\\2015bigdata\\riii\\samples\\*.txt'):
try :
    for i in range(0,5):
        print list[i]
except Exception, err:
    print traceback.format_exc()
    
    


1
3
4
5
Traceback (most recent call last):
  File "<ipython-input-36-785b43550fc1>", line 7, in <module>
    print list[i]
IndexError: list index out of range

